In [22]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Lasso
import re

In [23]:
df = pd.read_csv("all/train.csv", encoding='latin-1')
y = df['Rating']
stemmer = SnowballStemmer(language='english') 
idf_vectorizer = TfidfVectorizer()
corpus = df['Review_Text']
X = idf_vectorizer.fit_transform(corpus) # преобразование векторов
reg = Ridge().fit(X, y) # обучение
lambdas = np.arange(0.1, 3.0, 0.2) # лямбды
lambda_errors = []
for val in lambdas:
    cv_error = -np.average(cross_val_score(Ridge(alpha=val), X, y, cv=5, scoring='neg_mean_squared_error')) # подсчет ошибки
    lambda_errors.append((val, np.average(cv_error)))
min_val, min_train_error = min(lambda_errors, key=lambda x: x[1]) 
best_reg = Ridge(alpha=min_val)
cv_error = -np.average(cross_val_score(best_reg, X, y, cv=5, scoring='neg_mean_squared_error')) # ошибка

print(cv_error)

216.7870389450331


In [24]:
lambdas = 2.0 ** np.arange(-1, 10, 0.2) # лямбды
for val in lambdas:
    cv_error = -np.average(cross_val_score(Lasso(alpha=val), X, y, cv=5, scoring='neg_mean_squared_error')) # подсчет ошибки
    lambda_errors.append((val, np.average(cv_error)))
min_val, min_train_error = min(lambda_errors, key=lambda x: x[1])
best_reg = Lasso(alpha=min_val) # лучшая ошибка
cv_error = -np.average(cross_val_score(best_reg, X, y, cv=5, scoring='neg_mean_squared_error'))
print(cv_error)

444.7033597183325


In [25]:
reg = LinearRegression()
reg.fit(X, y) 
cv_error = -np.average(cross_val_score(LinearRegression(), X, y, cv=5, scoring='neg_mean_squared_error'))  # просто построение линейной регрессии
print(cv_error)

759.979824854634


In [26]:
vectorizer = CountVectorizer() # подсчет COuntvectoriez
X = vectorizer.fit_transform(corpus) 
reg = Ridge().fit(X, y) # используем L2-регуляризацию
lambdas = np.arange(0.1, 3.0, 0.2)
lambda_errors = []
for val in lambdas:
    cv_error = -np.average(cross_val_score(Ridge(alpha=val), X, y, cv=5, scoring='neg_mean_squared_error')) # считаем ошибку
    lambda_errors.append((val, np.average(cv_error)))
min_val, min_train_error = min(lambda_errors, key=lambda x: x[1]) # минимальная ошибка
best_reg = Ridge(alpha=min_val)
cv_error = -np.average(cross_val_score(best_reg, X, y, cv=5, scoring='neg_mean_squared_error'))

print(cv_error)

278.01894308972857


In [27]:
df = pd.read_csv("all/train.csv", encoding='latin-1')
y = df['Rating']
for i in range(len(df)):
    sent = df['Review_Text'][i]
    string = ""
    exp = r'(?<!\w\.\w.)(?<![А-Я][а-я]\.)(?<=\.|\?)\s' # выражение чтобы расплитить слова
    tmp_string = re.split(exp, sent)
    for sent in tmp_string:
        a = [stemmer.stem(word) for word in sent.split()] # удаление не нужных слов
        for x in a:
            string += x
            string += " " # преобразуем все оставщееся обратно в отзыв
    df['Review_Text'][i] = string
corpus = df['Review_Text']
X = idf_vectorizer.fit_transform(corpus) # делаем матрицу
reg = Ridge().fit(X, y)  
lambdas = np.arange(0.1, 3.0, 0.2)
lambda_errors = []
for val in lambdas:
    cv_error = -np.average(cross_val_score(Ridge(alpha=val), X, y, cv=5, scoring='neg_mean_squared_error')) # снова строим с L2
    lambda_errors.append((val, np.average(cv_error)))
min_val, min_train_error = min(lambda_errors, key=lambda x: x[1])
best_reg = Ridge(alpha=min_val)
cv_error = -np.average(cross_val_score(best_reg, X, y, cv=5, scoring='neg_mean_squared_error'))

print(cv_error)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


219.60910371527885


In [28]:
df = pd.read_csv("all/train.csv", encoding='latin-1')
expr = r'\W'
parser=re.compile(expr)
y = df['Rating']
f = open('positive-words.txt')
positive = f.read()
s = open('negative-words.txt', 'r', encoding="ISO-8859-1");
negative = s.read()
X = np.array([0 for i in range(len(df))])
for i in range(len(df)):
    sent = df['Review_Text'][i]
    sent = sent.lower() # приводим все к маленьким
    tmp_string = parser.sub(r' ', sent) # приводим к массиве чтобы можно было искать
    tmp_string = tmp_string.split()
    count = 0
    for x in tmp_string:
        if x in positive:
            count += 2.6 # считаем число у каждой строки
        if x in negative:
            count -= 1
    X[i] = count
X = X.reshape(-1, 1) 
reg = Ridge().fit(X, y) # строим  Линейную регрессию L2
lambdas = np.arange(0.1, 3.0, 0.2)
lambda_errors = []
for val in lambdas:
    cv_error = -np.average(cross_val_score(Ridge(alpha=val), X, y, cv=5, scoring='neg_mean_squared_error')) # считаем ошибку
    lambda_errors.append((val, np.average(cv_error)))
min_val, min_train_error = min(lambda_errors, key=lambda x: x[1])
best_reg = Ridge(alpha=min_val)
cv_error = -np.average(cross_val_score(best_reg, X, y, cv=5, scoring='neg_mean_squared_error'))

print(cv_error)

445.66108072919735


                                            Отчет
1. Я преобразовала отзывы в матрицу(idf_vectorizer) и построила линейную регрессию с L2-регуляризацией.
2. Я попробовала вместо L2 регуляризации сделать L1, результат ухудшился
3. Я решила вообще убрать регуляризации. 
4. Я преобразовала  отзывы в матрицу(сount_vectorizer) и построила линейную регрессию с L2-регуляризацией
5. Я решила убрать stop_words и преобразовала отзывы в матрицу(idf_vectorizer), и построила линейную регрессию с L2-регуляризацией.
6. Я попробовала использовать positive и negative words, добавляя за каждое позитивное слово что-тo и вычитая за каждое негативное слово что-то.

В результате лучшая ошибка была в (1).
